In [1]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

import glob as glob
import os as os
import subprocess as sp
import shutil as shutil
from itertools import compress

import parmed as pmd
import pytraj as pt

In [2]:
systems = """./a-bam-p
./a-bam-s
./a-but-p
./a-but-s
./a-cbu-p
./a-chp-p
./a-cbu-s
./a-chp-s
./a-cpe-p
./a-coc-p
./a-coc-s
./a-cpe-s
./a-hep-p
./a-ham-s
./a-ham-p
./a-hep-s
./a-hp6-p
./a-hex-p
./a-hex-s
./a-hp6-s
./a-hx2-p
./a-hpa-s
./a-hpa-p
./a-hx2-s
./a-mba-p
./a-hx3-s
./a-hx3-p
./a-mba-s
./a-mhp-p
./a-mha-p
./a-mha-s
./a-mhp-s
./a-nmh-p
./a-nmb-p
./a-nmb-s
./a-nmh-s
./a-oct-p
./a-oam-p
./a-oam-s
./a-oct-s
./a-pnt-p
./a-pam-p
./a-pam-s
./a-pnt-s
./b-ben-s
./a-xxxx-s
./b-ben-p
./b-cbu-p
./b-cbu-s
./b-chp-s
./b-chp-p
./b-coc-s
./b-coc-p
./b-cpe-s
./b-cpe-p
./b-ham-s
./b-ham-p
./b-hep-s
./b-hep-p
./b-hex-p
./b-hex-s
./b-m4c-s
./b-m4c-p
./b-m4t-p
./b-m4t-s
./b-mch-s
./b-mha-s
./b-mha-p
./b-mch-p
./b-mo3-s
./b-mo4-p
./b-mo4-s
./b-mo3-p
./b-mp3-s
./b-mp4-s
./b-mp4-p
./b-mp3-p
./b-oam-s
./b-pb3-s
./b-pb3-p
./b-oam-p
./b-pb4-s
./b-pha-s
./b-pb4-p
./b-pha-p
./b-pnt-s
./b-pnt-p"""
systems = systems.split("\n")
systems = [i[2:] for i in systems]
systems = [i for i in systems if "xxxx" not in i]

In [39]:
files = ["equil.rst7", "equil-centered.inpcrd"]

In [45]:
for system in systems:
    pull_windows = glob.glob(os.path.join("systems", system, "smirnoff") + "/p*")
    for window in pull_windows:
        for file in files:
            traj = pt.load(os.path.join(window, file),
                           top = os.path.join(window, "smirnoff.prmtop"))
            guest = window.split("/")[1].split("-")[1].upper()
            guest_host = pt.distance(traj, f":{guest} :MGO")
            guest_dumm = pt.distance(traj, f":{guest} @Pb")

            if guest_dumm < guest_host:
                print(window, file, guest_host, guest_dumm)

systems/a-bam-p/smirnoff/p028 equil-centered.inpcrd [ 39.22817972] [ 27.54205479]
systems/a-bam-p/smirnoff/p029 equil-centered.inpcrd [ 38.40167951] [ 27.32494454]
systems/a-bam-p/smirnoff/p030 equil-centered.inpcrd [ 38.23429629] [ 27.0282075]
systems/a-bam-p/smirnoff/p031 equil-centered.inpcrd [ 37.99449967] [ 26.84507411]
systems/a-bam-p/smirnoff/p032 equil-centered.inpcrd [ 37.84301242] [ 26.06401605]
systems/a-bam-p/smirnoff/p033 equil-centered.inpcrd [ 36.70312784] [ 25.37131907]
systems/a-bam-p/smirnoff/p034 equil-centered.inpcrd [ 37.38773705] [ 25.68933991]
systems/a-bam-p/smirnoff/p035 equil-centered.inpcrd [ 35.52572063] [ 24.47402839]
systems/a-bam-p/smirnoff/p036 equil-centered.inpcrd [ 35.72039338] [ 25.30962142]
systems/a-bam-p/smirnoff/p037 equil-centered.inpcrd [ 36.30561929] [ 24.53251075]
systems/a-bam-p/smirnoff/p038 equil-centered.inpcrd [ 35.2117281] [ 23.311207]
systems/a-bam-p/smirnoff/p039 equil-centered.inpcrd [ 35.07436779] [ 23.71986859]
systems/a-bam-p/smir

Now run without imaging in `cpptraj` and see if the problem has resolved itself.

In [61]:
def center_no_image(prmtop, inpcrd, mask, cwd="./"):
    trajout = inpcrd.split(".")[0] + "-centered-no-image.inpcrd"
    cpptraj = f"""
    parm {prmtop}
    trajin {inpcrd}
    center {mask} origin
    trajout {trajout} restart
    """
    with open(os.path.join(cwd, "center.in"), "w") as f:
        f.write(cpptraj)
    sp.call("cpptraj -i center.in > center.out", shell=True, cwd=cwd)

In [63]:
for system in systems:
    pull_windows = glob.glob(os.path.join("systems", system, "smirnoff") + "/p*")
    for window in pull_windows:
        center_no_image("smirnoff.prmtop", "equil.rst7", "@Pb", cwd=window)
        traj = pt.load(
            os.path.join(window, "equil-centered-no-image.inpcrd"),
            top=os.path.join(window, "smirnoff.prmtop"),
        )
        guest = window.split("/")[1].split("-")[1].upper()
        guest_host = pt.distance(traj, f":{guest} :MGO")
        guest_dumm = pt.distance(traj, f":{guest} @Pb")

        if guest_dumm < guest_host:
            print(window, file, guest_host, guest_dumm)


Okay, I think things are fixed.

In [ ]:
githu